In [0]:
import gym
import numpy as np
import random
from tensorflow.python.keras.layers import  Dense, Flatten, Activation,Conv2D
from tensorflow.python.keras.models import Sequential
import os
from tensorflow.python.keras.models import load_model
import matplotlib.pyplot as plt
import cv2
from collections import deque
from tensorflow.python.keras.optimizers import RMSprop
from google.colab import files


In [0]:
!pip install gym[atari]

In [0]:
env = gym.make("Breakout-v0")

In [0]:
input_shape = (84, 84, 1)
model = Sequential()
model.add(Conv2D(32, 8, strides=(4, 4), padding="valid", activation="relu", input_shape=input_shape))
model.add(Conv2D(64, 4, strides=(2, 2), padding="valid", activation="relu")) 
model.add(Conv2D(64, 3, strides=(1, 1), padding="valid", activation="relu", input_shape=input_shape))
model.add(Flatten())                              
model.add(Dense(512, activation="relu"))                             
model.add(Dense(env.action_space.n))                              
model.compile(loss="mean_squared_error", optimizer=RMSprop(lr=0.00025, rho=0.95, epsilon=0.01), metrics=["accuracy"])
                           
                                             
                                             
                                                         
                              
                                                           
                              
                              
                              
                              
                                                           
                              
                              
                              
                                    
                     

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
files.upload()
if os.path.isfile('bo_model16502400.h5'):
  print("hi")
  model = load_model("bo_model16502400.h5")

Saving bo_model16502400.h5 to bo_model16502400.h5
hi
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
gamma = 1.0
epsilon = 1.0
m = []
num_actions = 6
num_frames = 3
blend = 3
image = deque(maxlen=3)


In [0]:
def blend_image(images, blend):
  avg_image = np.expand_dims(np.zeros((84, 84, 1), np.float64), axis=0)

  for image in images:
    avg_image += image
        
  if len(images) < blend:
    return avg_image / len(images)
  else:
    return avg_image / blend

In [0]:
def process_image(frame):
  img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  img = cv2.resize(img, (84,84))

  return np.expand_dims(img.reshape(84, 84, 1), axis=0)





In [0]:
for i in range(5000):
    state = env.reset()
    #print(state.shape)
    state = process_image(state)
    image.append(state)
    for t in range(20000):
      state = blend_image(image, blend)
      if np.random.rand() <= epsilon:
          action = random.randrange(env.action_space.n)
      else:
          action = np.argmax(model.predict(state))
      new_state, reward, done, observation = env.step(action)
      new_state = process_image(new_state)
      image.append(new_state)
      new_state = blend_image(image, blend)
      tot = reward + gamma * np.max(model.predict(new_state))
      p = model.predict(state)[0]
      p[action] = tot
      model.fit(state, p.reshape(-1, env.action_space.n), epochs=1, verbose=0)
      m.append((state, action, reward, new_state, done))
      state = new_state
      if done:
          print("Episode : {}, Score: {}".format(i, t))
          break
      if len(m) == 50000:
          del m[:5000]
    if epsilon > 0.01:
        epsilon *= 0.999
    # print(len(m))
    if len(m) > 64:
        for state, action, reward, next_state, done in random.sample(m, 64):
            tot = reward
            if not done:
                tot = reward + gamma * np.max(model.predict(next_state))

            p = model.predict(state)[0]
            p[action] = tot
            model.fit(state, p.reshape(-1, env.action_space.n), epochs=1, verbose=0)
    if i % 150 == 0:
        model.save("bo_model1650" + str(2400+i) + ".h5")

Episode : 0, Score: 236
Episode : 1, Score: 173
Episode : 2, Score: 249
Episode : 3, Score: 239
Episode : 4, Score: 215
Episode : 5, Score: 211
Episode : 6, Score: 365
Episode : 7, Score: 208
Episode : 8, Score: 184
Episode : 9, Score: 184
Episode : 10, Score: 414
Episode : 11, Score: 174
Episode : 12, Score: 171
Episode : 13, Score: 179
Episode : 14, Score: 183
Episode : 15, Score: 169
Episode : 16, Score: 440
Episode : 17, Score: 171
Episode : 18, Score: 343
Episode : 19, Score: 234
Episode : 20, Score: 421
Episode : 21, Score: 171
Episode : 22, Score: 169
Episode : 23, Score: 177
Episode : 24, Score: 219
Episode : 25, Score: 206
Episode : 26, Score: 178
Episode : 27, Score: 319
Episode : 28, Score: 183
Episode : 29, Score: 231
Episode : 30, Score: 314
Episode : 31, Score: 205
Episode : 32, Score: 197
Episode : 33, Score: 179
Episode : 34, Score: 291
Episode : 35, Score: 173
Episode : 36, Score: 301
Episode : 37, Score: 274
Episode : 38, Score: 248
Episode : 39, Score: 373
Episode : 

KeyboardInterrupt: ignored

In [0]:
--------------------------------------------------------------------------------------------------------------------------------------------------------state = env.reset()
print(env.action_space.n)

In [0]:
!ls



sample_data


In [0]:
files.download('bo_model16502100.h5')

NameError: ignored